In [1]:
import pandas as pd
import numpy as np
from ark_tweet_nlp import CMUTweetTagger
from tqdm import tqdm_notebook as tqdm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import pickle
import copy
import re

The processed data contains pre-processed text to search for patterns. This time we don't need the neighborhoods for hashtag information as the hashtags don't contain any negative-PI-related information in the McDonald's data set.

In [2]:
"""Loading the tagged data and the 5-words neighborhoods """
data = pickle.load(open('./processed_MCD.b', 'rb'))

In [3]:
"""Preparing the data"""
data['tokens'] = data.text.apply(lambda x: word_tokenize(x.lower()))

In [6]:
"""Pattern flagging"""

"""Structuring the words into stand alone identifiers, negative paired identifiers, negative indicators, BA indicators, and other fluff"""
stand_alone = ['boycott','bycott','boycot','bycot','avoid']
neg_paired = ['eat','get','have','buy']
neg_ind = ['never','not','dont','shouldnt','no','wont','wouldnt']
lmtzr = WordNetLemmatizer()

ys = list(data.pi)

pred_2 = []
for k in tqdm(range(data.shape[0]), desc='Going through the neighborhoods for 2 conditions'):
    label = False
    for i in range(len(data.tokens.iloc[k])):
        if lmtzr.lemmatize(data.tokens.iloc[k][i],'v') in stand_alone:
            label = True
        if data.tokens.iloc[k][i] in neg_ind:
            for j in range(i,min(len(data.tokens.iloc[k]),i+4)):
                if lmtzr.lemmatize(data.tokens.iloc[k][j],'v') in neg_paired:
                    label = True
                    break
        if label:
            pred_2.append(1)
            break
    """Labeling as a non negative PI tweet if no condition is met"""
    if not label:
        pred_2.append(0)
        
print('Using 2 conditions : negative PI verb, negation word + positive PI verb with hashtags')
print('F1 score obtained through the fully supervised method : {}'.format(f1_score(y_true=ys, y_pred=pred_2)))
print('Precision obtained through the fully supervised method : {}'.format(precision_score(y_true=ys, y_pred=pred_2)))
print('Recall score obtained through the fully supervised method : {}'.format(recall_score(y_true=ys, y_pred=pred_2)))


Using 2 conditions : negative PI verb, negation word + positive PI verb with hashtags
F1 score obtained through the fully supervised method : 0.49233656553018457
Precision obtained through the fully supervised method : 0.4607728337236534
Recall score obtained through the fully supervised method : 0.5285426460711887
